In [1]:
import os
import sys
import pickle
from pyproj import CRS

from network_wrangler import RoadwayNetwork
from network_wrangler import TransitNetwork
from network_wrangler import Scenario
from network_wrangler import WranglerLogger

from lasso import Parameters
from lasso import metcouncil
from lasso import ModelRoadwayNetwork
from lasso import StandardTransit
from lasso import util

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import logging
logger = logging.getLogger("WranglerLogger")
logger.handlers[0].stream = sys.stdout
logger.setLevel(logging.INFO)

# remote i/o

In [4]:
input_dir = os.path.join("C:/MetC_LassoData/data/processed/version_00")
lasso_dir = os.path.join("C:/Lasso")
output_dir = os.path.join("C:/MetC_LassoData/data/processed/version_01")

In [5]:
project_card_dir = os.path.join("C:/project_card_registry/projects/BaseCorrections")

In [6]:
parameters = Parameters(lasso_base_dir = lasso_dir)

2022-07-08 12:40:28, INFO: Lasso base directory set as: C:/Lasso
2022-07-08 12:40:28, INFO: Lasso base directory set as: C:/Lasso


# Load Version00

In [7]:
link_file = os.path.join(input_dir, 'standard_networks', 'links.json')
node_file = os.path.join(input_dir, 'standard_networks', 'nodes.geojson')
shape_file = os.path.join(input_dir, 'standard_networks', 'shapes.geojson')

roadway_net = RoadwayNetwork.read(link_file = link_file, node_file = node_file, shape_file = shape_file)

2022-07-08 12:40:28, INFO: Reading from following files:
-C:/MetC_LassoData/data/processed/version_00\standard_networks\links.json
-C:/MetC_LassoData/data/processed/version_00\standard_networks\nodes.geojson
-C:/MetC_LassoData/data/processed/version_00\standard_networks\shapes.geojson.
2022-07-08 12:40:28, INFO: Reading from following files:
-C:/MetC_LassoData/data/processed/version_00\standard_networks\links.json
-C:/MetC_LassoData/data/processed/version_00\standard_networks\nodes.geojson
-C:/MetC_LassoData/data/processed/version_00\standard_networks\shapes.geojson.
2022-07-08 12:44:39, INFO: Read 1061566 links from C:/MetC_LassoData/data/processed/version_00\standard_networks\links.json
2022-07-08 12:44:39, INFO: Read 1061566 links from C:/MetC_LassoData/data/processed/version_00\standard_networks\links.json
2022-07-08 12:44:39, INFO: Read 414131 nodes from C:/MetC_LassoData/data/processed/version_00\standard_networks\nodes.geojson
2022-07-08 12:44:39, INFO: Read 414131 nodes from C:

In [8]:
transit_net = TransitNetwork.read(feed_path = os.path.join(input_dir, 'standard_networks', 'transit'))

2022-07-08 12:47:20, INFO: Read in transit feed from: C:/MetC_LassoData/data/processed/version_00\standard_networks\transit
2022-07-08 12:47:20, INFO: Read in transit feed from: C:/MetC_LassoData/data/processed/version_00\standard_networks\transit
2022-07-08 12:47:20, INFO: Removing calendar.txt from transit network config because file not found
2022-07-08 12:47:20, INFO: Removing calendar.txt from transit network config because file not found
2022-07-08 12:47:20, INFO: Removing calendar_dates.txt from transit network config because file not found
2022-07-08 12:47:20, INFO: Removing calendar_dates.txt from transit network config because file not found
2022-07-08 12:47:21, INFO: Removing fare_attributes.txt from transit network config because file not found
2022-07-08 12:47:21, INFO: Removing fare_attributes.txt from transit network config because file not found
2022-07-08 12:47:21, INFO: Removing fare_rules.txt from transit network config because file not found
2022-07-08 12:47:21, INF

In [9]:
roadway_net.links_df = roadway_net.links_df[roadway_net.links_df.A != roadway_net.links_df.B]

### Attribute the Network

In [10]:
roadway_net.links_df = roadway_net.links_df.drop('lanes', axis = 1)

In [11]:
roadway_net.links_df['bus_only'] = 0

In [12]:
roadway_net.links_df.drive_access.value_counts()

True     656665
False    404892
Name: drive_access, dtype: int64

In [13]:
r_net = metcouncil.calculate_number_of_lanes_from_reviewed_network(
    roadway_net=roadway_net,
    parameters=parameters,
)
r_net.links_df.lanes.value_counts()

2022-07-08 12:47:27, INFO: Calculating Number of Lanes as network variable: 'lanes'
2022-07-08 12:47:27, INFO: Calculating Number of Lanes as network variable: 'lanes'
2022-07-08 12:47:27, INFO: Lasso base directory set as: C:/Lasso
2022-07-08 12:47:27, INFO: Lasso base directory set as: C:/Lasso
2022-07-08 12:47:27, INFO: Calculating Centroid Connector and adding as roadway network variable: centroidconnect
2022-07-08 12:47:27, INFO: Calculating Centroid Connector and adding as roadway network variable: centroidconnect
2022-07-08 12:47:27, INFO: Finished calculating centroid connector variable: centroidconnect
2022-07-08 12:47:27, INFO: Finished calculating centroid connector variable: centroidconnect
2022-07-08 12:48:49, INFO: Finished calculating number of lanes to: lanes
2022-07-08 12:48:49, INFO: Finished calculating number of lanes to: lanes


1.0    1026496
2.0      32020
3.0       2595
4.0        369
5.0         71
6.0          5
7.0          1
Name: lanes, dtype: int64

In [14]:
r_net.links_df.roadway.value_counts()

residential       420553
service           264058
footway           117014
tertiary          113519
cycleway           83868
secondary          36639
primary            14067
motorway_link       3604
trunk               3051
motorway            2757
secondary_link       839
trunk_link           601
tertiary_link        538
primary_link         449
Name: roadway, dtype: int64

In [15]:
r_net = metcouncil.calculate_assign_group_and_roadway_class_from_reviewed_network(
        roadway_net=r_net,
        parameters=parameters,
)
r_net.links_df.assign_group.value_counts()

2022-07-08 12:48:51, INFO: Calculating Assignment Group and Roadway Class as network variables: 'assign_group' and 'roadway_class'
2022-07-08 12:48:51, INFO: Calculating Assignment Group and Roadway Class as network variables: 'assign_group' and 'roadway_class'
2022-07-08 12:48:51, INFO: Centroid Connector Variable 'centroidconnect' already in network. Returning without overwriting.
2022-07-08 12:48:51, INFO: Centroid Connector Variable 'centroidconnect' already in network. Returning without overwriting.
2022-07-08 12:52:21, INFO: Finished calculating assignment group variable assign_group and roadway class variable roadway_class
2022-07-08 12:52:21, INFO: Finished calculating assignment group variable assign_group and roadway class variable roadway_class


50.0     294095
103.0    204010
102.0    117014
7.0      101584
101.0     83868
6.0       76802
5.0       15025
15.0       3045
3.0        2590
1.0        2455
4.0        2036
2.0        1258
11.0        153
Name: assign_group, dtype: int64

In [16]:
r_net.links_df.roadway_class.value_counts()

101.0    404892
50.0     304018
40.0     124841
30.0      42279
20.0      20590
60.0       5416
10.0       1780
70.0        119
Name: roadway_class, dtype: int64

### Add Rail links and nodes

In [17]:
r_net = metcouncil.add_rail_links_and_nodes(
    roadway_network = r_net,
    parameters = parameters,
    rail_links_file = os.path.join(input_dir, 'standard_networks', 'rail_links.geojson'),
    rail_nodes_file = os.path.join(input_dir, 'standard_networks', 'rail_nodes.geojson'),
)

2022-07-08 12:52:22, INFO: Adding centroid and centroid connector to standard network
2022-07-08 12:52:22, INFO: Adding centroid and centroid connector to standard network
2022-07-08 12:52:22, INFO: Lasso base directory set as: C:/Lasso
2022-07-08 12:52:22, INFO: Lasso base directory set as: C:/Lasso
2022-07-08 12:52:24, INFO: Finished adding rail links and nodes connectors
2022-07-08 12:52:24, INFO: Finished adding rail links and nodes connectors


In [18]:
# check if missing IDs
# rail nodes does not have osm IDs, they have shst IDs
# rail links does not have osm IDs, they have shst IDs

# if node missing shst id
print(r_net.nodes_df.shst_node_id.isnull().sum())
print(r_net.nodes_df.shst_node_id.nunique())
# if node missing osm id
print(r_net.nodes_df.osm_node_id.isnull().sum())
# if node missing osm id
print(r_net.nodes_df.osm_node_id.dtype)
print(r_net.nodes_df[r_net.nodes_df.osm_node_id == 0])
print(r_net.nodes_df.osm_node_id.nunique())
# if node missing model node id
print(r_net.nodes_df.model_node_id.nunique())

# if link missing 
print(r_net.links_df.shstReferenceId.isnull().sum())
print(r_net.links_df.shstReferenceId.nunique())
print(r_net.links_df.model_link_id.nunique())

# if link missing node id
print(r_net.links_df.fromIntersectionId.isnull().sum())
print(r_net.links_df.toIntersectionId.isnull().sum())
print(r_net.links_df.u.isnull().sum())
print(r_net.links_df[r_net.links_df.u == 0])
print(r_net.links_df.v.isnull().sum())
print(r_net.links_df[r_net.links_df.v == 0])

0
414222
1995
float64
Empty GeoDataFrame
Columns: [osm_node_id, shst_node_id, county, drive_access, walk_access, bike_access, model_node_id, rail_only, geometry, X, Y]
Index: []
412227
414222
0
1061647
1061647
0
0
5072
Empty GeoDataFrame
Columns: [shstReferenceId, id, shstGeometryId, fromIntersectionId, toIntersectionId, u, v, nodeIds, wayId, roadClass, oneWay, roundabout, link, oneway, ref, name, highway, service, width, maxspeed, access, junction, bridge, tunnel, landuse, area, key, forward, backReferenceId, metadata, source, roadway, drive_access, walk_access, bike_access, county, length, A, B, model_link_id, locationReferences, rail_only, geometry, bus_only, centroidconnect, lanes, assign_group, roadway_class]
Index: []

[0 rows x 48 columns]
5072
Empty GeoDataFrame
Columns: [shstReferenceId, id, shstGeometryId, fromIntersectionId, toIntersectionId, u, v, nodeIds, wayId, roadClass, oneWay, roundabout, link, oneway, ref, name, highway, service, width, maxspeed, access, junction, bri

# Create Scenario 00

In [19]:
base_scenario = {"road_net": r_net, "transit_net": transit_net}

In [20]:
version_00_scenario = Scenario.create_scenario(base_scenario = base_scenario)

2022-07-08 12:52:28, INFO: Creating Scenario
2022-07-08 12:52:28, INFO: Creating Scenario


In [21]:
working_scenario_filename = os.path.join(output_dir, 'version_00_standard_network.pickle')
pickle.dump(version_00_scenario, open(working_scenario_filename, 'wb'))

In [22]:
r_net.links_df.shape

(1061647, 48)

In [23]:
r_net.links_df.model_link_id.max()

1061656

In [24]:
r_net.nodes_df.shape

(414222, 11)

In [25]:
r_net.nodes_df.model_node_id.max()

417322

# Create Scenario 01

In [26]:
version_01_scenario = Scenario.create_scenario(
    base_scenario=version_00_scenario,
    card_directory = project_card_dir,
    validate_project_cards=False
)

2022-07-08 12:53:52, INFO: Creating Scenario
2022-07-08 12:53:52, INFO: Creating Scenario


In [27]:
print(project_card_dir)

C:/project_card_registry/projects/BaseCorrections


In [28]:
version_01_scenario.apply_all_projects()

2022-07-08 12:53:56, INFO: Applying correct year 2018 assignment group and roadway class
2022-07-08 12:53:56, INFO: Applying correct year 2018 assignment group and roadway class
2022-07-08 12:53:56, INFO: Applying correct year 2018 assignment group and roadway class
2022-07-08 12:53:56, INFO: Applying correct year 2018 assignment group and roadway class
2022-07-08 12:53:56, INFO: Applying Project to Roadway Network: correct year 2018 assignment group and roadway class
2022-07-08 12:53:56, INFO: Applying Project to Roadway Network: correct year 2018 assignment group and roadway class
2022-07-08 12:53:56, INFO: Applying correct year 2018 assignment group
2022-07-08 12:53:56, INFO: Applying correct year 2018 assignment group
2022-07-08 12:53:56, INFO: Applying correct year 2018 assignment group
2022-07-08 12:53:56, INFO: Applying correct year 2018 assignment group
2022-07-08 12:53:56, INFO: Applying Project to Roadway Network: correct year 2018 assignment group
2022-07-08 12:53:56, INFO: 

C:\ProgramData\Anaconda3\envs\lasso_env\lib\site-packages\geopandas\geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


2022-07-08 12:54:05, INFO: Applying correct year 2018 assignment group
2022-07-08 12:54:05, INFO: Applying correct year 2018 assignment group
2022-07-08 12:54:05, INFO: Applying Project to Roadway Network: correct year 2018 assignment group
2022-07-08 12:54:05, INFO: Applying Project to Roadway Network: correct year 2018 assignment group


C:\ProgramData\Anaconda3\envs\lasso_env\lib\site-packages\geopandas\geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


2022-07-08 12:54:14, INFO: Applying correct year 2018 assignment group
2022-07-08 12:54:14, INFO: Applying correct year 2018 assignment group
2022-07-08 12:54:14, INFO: Applying Project to Roadway Network: correct year 2018 assignment group
2022-07-08 12:54:14, INFO: Applying Project to Roadway Network: correct year 2018 assignment group


...

C:\ProgramData\Anaconda3\envs\lasso_env\lib\site-packages\geopandas\geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


2022-07-08 13:20:09, INFO: Applying network cleanup 1 asgngrp 1
2022-07-08 13:20:09, INFO: Applying network cleanup 1 asgngrp 1
2022-07-08 13:20:09, INFO: Applying network cleanup 1 asgngrp 1
2022-07-08 13:20:09, INFO: Applying network cleanup 1 asgngrp 1
2022-07-08 13:20:09, INFO: Applying Project to Roadway Network: network cleanup 1 asgngrp 1
2022-07-08 13:20:09, INFO: Applying Project to Roadway Network: network cleanup 1 asgngrp 1


C:\ProgramData\Anaconda3\envs\lasso_env\lib\site-packages\geopandas\geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


2022-07-08 13:20:15, INFO: Applying Add walk and bike attributes
2022-07-08 13:20:15, INFO: Applying Add walk and bike attributes
2022-07-08 13:20:15, INFO: Applying Project to Roadway Network: Add walk and bike attributes
2022-07-08 13:20:15, INFO: Applying Project to Roadway Network: Add walk and bike attributes


C:\ProgramData\Anaconda3\envs\lasso_env\lib\site-packages\geopandas\geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [29]:
version_01_scenario.applied_projects

['correct year 2018 assignment group and roadway class',
 'correct year 2018 assignment group',
 'network cleanup remove link',
 'Manual Changes to Assignment Group and Roadway Class',
 'network cleanup 1 lanes 7',
 'network cleanup 1 lanes 6',
 'network cleanup 1 lanes 5',
 'network cleanup 1 lanes 4',
 'network cleanup 1 lanes 3',
 'network cleanup 1 lanes 2e',
 'network cleanup 1 lanes 2d part 2',
 'network cleanup 1 lanes 2d',
 'network cleanup 1 lanes 2c part 2',
 'network cleanup 1 lanes 2c',
 'network cleanup 1 lanes 2b',
 'network cleanup 1 lanes 2a',
 'network cleanup 1 lanes 1 h',
 'network cleanup 1 lanes 1 g',
 'network cleanup 1 lanes 1 f part 2',
 'network cleanup 1 lanes 1 f',
 'network cleanup 1 lanes 1 e part 2',
 'network cleanup 1 lanes 1 e',
 'network cleanup 1 lanes 1 d part 2',
 'network cleanup 1 lanes 1 d',
 'network cleanup 1 lanes 1 c part 2',
 'network cleanup 1 lanes 1 c',
 'network cleanup 1 lanes 1 b part 2',
 'network cleanup 1 lanes 1 b',
 'network clean

In [30]:
version_01_scenario.road_net.links_df.shape

(1061709, 52)

In [31]:
version_01_scenario.road_net.links_df.model_link_id.nunique()

1061709

In [32]:
version_01_scenario.road_net.nodes_df.shape

(414253, 11)

In [33]:
version_01_scenario.road_net.nodes_df.model_node_id.nunique()

414253

In [34]:
version_01_scenario.road_net.links_df['access'] = version_01_scenario.road_net.links_df['access'].fillna('')
version_01_scenario.road_net.links_df['access'] = version_01_scenario.road_net.links_df['access'].apply(
    lambda x: util.shorten_name(x)
)

In [35]:
version_01_scenario.road_net.links_df['name'] = version_01_scenario.road_net.links_df['name'].fillna('')
version_01_scenario.road_net.links_df['name'] = version_01_scenario.road_net.links_df['name'].apply(
    lambda x: util.shorten_name(x)
)

# Save version 01 standard networks

In [36]:
version_01_scenario.road_net.write(path = os.path.join(output_dir, 'standard_networks'))

# Save version 01 standard network as pickle file

In [37]:
working_scenario_filename = os.path.join(output_dir, 'version_01_standard_network.pickle')
pickle.dump(version_01_scenario, open(working_scenario_filename, 'wb'))

# Make Travel Model Network

In [38]:
m_net = ModelRoadwayNetwork.from_RoadwayNetwork(
    roadway_network_object = version_01_scenario.road_net, 
    parameters = parameters
)

2022-07-08 13:44:05, INFO: Lasso base directory set as: C:/Lasso
2022-07-08 13:44:05, INFO: Lasso base directory set as: C:/Lasso


### Add centroid and centroid connectors

In [39]:
m_net = metcouncil.add_centroid_and_centroid_connector(
    roadway_network = m_net,
    parameters = parameters,
    centroid_file = os.path.join(input_dir, 'standard_networks', 'centroid_node.pickle'),
    centroid_connector_link_file = os.path.join(input_dir, 'standard_networks', 'cc_link.pickle'),
    centroid_connector_shape_file = os.path.join(input_dir, 'standard_networks', 'cc_shape.pickle'),
)

2022-07-08 13:44:05, INFO: Adding centroid and centroid connector to standard network
2022-07-08 13:44:05, INFO: Adding centroid and centroid connector to standard network
2022-07-08 13:44:05, INFO: Lasso base directory set as: C:/Lasso
2022-07-08 13:44:05, INFO: Lasso base directory set as: C:/Lasso
2022-07-08 13:44:10, INFO: Finished adding centroid and centroid connectors
2022-07-08 13:44:10, INFO: Finished adding centroid and centroid connectors


### Add Rail access and egress links

In [40]:
m_net = metcouncil.add_rail_ae_connections(
    m_net,
    parameters
)

2022-07-08 13:44:11, INFO: Creating rail access and egress connection links
2022-07-08 13:44:11, INFO: Creating rail access and egress connection links


C:\ProgramData\Anaconda3\envs\lasso_env\lib\site-packages\pandas\core\dtypes\cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [41]:
m_net.roadway_standard_to_met_council_network()

2022-07-08 13:44:47, INFO: Renaming roadway attributes to be consistent with what metcouncil's model is expecting
2022-07-08 13:44:47, INFO: Renaming roadway attributes to be consistent with what metcouncil's model is expecting
2022-07-08 13:44:47, INFO: Didn't detect managed lanes in network.
2022-07-08 13:44:47, INFO: Didn't detect managed lanes in network.
2022-07-08 13:44:47, INFO: Creating calculated roadway variables.
2022-07-08 13:44:47, INFO: Creating calculated roadway variables.
2022-07-08 13:44:47, INFO: Area Type Variable 'area_type' already in network. Returning without overwriting.
2022-07-08 13:44:47, INFO: Area Type Variable 'area_type' already in network. Returning without overwriting.
2022-07-08 13:44:47, INFO: Overwriting existing County Variable 'county' already in network
2022-07-08 13:44:47, INFO: Overwriting existing County Variable 'county' already in network
2022-07-08 13:44:47, INFO: Adding roadway network variable for county using a spatial join with: C:/Lass

c:\lasso\lasso\roadway.py:288: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroids_gdf["geometry"] = centroids_gdf["geometry"].centroid
c:\lasso\lasso\roadway.py:1225: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  self.create_calculated_variables()
c:\lasso\lasso\roadway.py:292: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: EPSG:4269

  joined_gdf = gpd.sjoin(centroids_gdf, county_gdf, how="left", op="intersects")
c:\lasso\lasso\roadway.py:1225: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  self.creat

2022-07-08 13:50:11, INFO: Finished Calculating county variable: county
2022-07-08 13:50:11, INFO: Finished Calculating county variable: county
2022-07-08 13:50:17, INFO: Calculating MPO as roadway network variable: mpo
2022-07-08 13:50:17, INFO: Calculating MPO as roadway network variable: mpo
2022-07-08 13:50:17, INFO: Finished calculating MPO variable: mpo
2022-07-08 13:50:17, INFO: Finished calculating MPO variable: mpo
2022-07-08 13:50:17, INFO: Adding Counts
2022-07-08 13:50:17, INFO: Adding Counts
2022-07-08 13:50:17, INFO: Adding Variable AADT using Shared Streets Reference from C:/Lasso\metcouncil_data\count_mn\mn_count_ShSt_API_match.csv
2022-07-08 13:50:17, INFO: Adding Variable AADT using Shared Streets Reference from C:/Lasso\metcouncil_data\count_mn\mn_count_ShSt_API_match.csv
2022-07-08 13:50:34, INFO: Added variable: AADT using Shared Streets Reference
2022-07-08 13:50:34, INFO: Added variable: AADT using Shared Streets Reference
2022-07-08 13:50:34, INFO: Adding Variab

2022-07-08 13:53:04, WARNING: Specified variable to split: price not in network variables: Index(['shstReferenceId', 'id', 'shstGeometryId', 'fromIntersectionId',
       'toIntersectionId', 'u', 'v', 'nodeIds', 'wayId', 'roadClass', 'oneWay',
       'roundabout', 'link', 'oneway', 'ref', 'name', 'highway', 'service',
       'width', 'maxspeed', 'access', 'junction', 'bridge', 'tunnel',
       'landuse', 'area', 'key', 'forward', 'backReferenceId', 'metadata',
       'source', 'roadway', 'drive_access', 'walk_access', 'bike_access',
       'county', 'length', 'A', 'B', 'model_link_id', 'locationReferences',
       'rail_only', 'geometry', 'bus_only', 'centroidconnect', 'lanes',
       'assign_group', 'roadway_class', 'area_type', 'ramp_flag', 'bike',
       'walk', 'mpo', 'AADT', 'count_AM', 'count_MD', 'count_PM', 'count_NT',
       'count_daily', 'count_year', 'segment_id', 'managed', 'distance',
       'trn_priority_AM', 'trn_priority_MD', 'trn_priority_PM',
       'trn_priority_NT',

In [42]:
# check if missing IDs
# centroids does not have osm and shst IDs
# centroid connectors does not have osm and shst IDs

# if node missing shst id
print(m_net.nodes_df.shst_node_id.isnull().sum())
print(m_net.nodes_df.shst_node_id.nunique())

# if node missing model node id
print(m_net.nodes_df.model_node_id.nunique())

# if link missing 
print(m_net.links_df.shstReferenceId.isnull().sum())
print(m_net.links_df.shstReferenceId.nunique())
print(m_net.links_df.model_link_id.nunique())

# if link missing node id
print(m_net.links_df.fromIntersectionId.isnull().sum())
print(m_net.links_df.toIntersectionId.isnull().sum())

0
414223
417283
0
1061648
1109533
0
0


In [43]:
m_net.nodes_df.shape

(417283, 11)

In [44]:
m_net.nodes_metcouncil_df.columns

Index(['osm_node_id', 'shst_node_id', 'drive_access', 'walk_access',
       'bike_access', 'N', 'rail_only', 'geometry', 'X', 'Y', 'county'],
      dtype='object')

In [45]:
m_net.links_metcouncil_df.columns

Index(['shstReferenceId', 'id', 'shstGeometryId', 'fromIntersectionId',
       'toIntersectionId', 'u', 'v', 'nodeIds', 'wayId', 'roadClass', 'oneWay',
       'roundabout', 'link', 'oneway', 'ref', 'name', 'highway', 'service',
       'width', 'maxspeed', 'access', 'junction', 'bridge', 'tunnel',
       'landuse', 'area', 'key', 'forward', 'backReferenceId', 'metadata',
       'source', 'roadway', 'drive_access', 'walk_access', 'bike_access',
       'county', 'length', 'A', 'B', 'model_link_id', 'locationReferences',
       'rail_only', 'bus_only', 'centroidconnect', 'lanes', 'assign_group',
       'roadway_class', 'area_type', 'ramp_flag', 'bike', 'walk', 'mpo',
       'AADT', 'count_AM', 'count_MD', 'count_PM', 'count_NT', 'count_daily',
       'count_year', 'segment_id', 'managed', 'distance', 'trn_priority_AM',
       'trn_priority_MD', 'trn_priority_PM', 'trn_priority_NT',
       'ttime_assert_AM', 'ttime_assert_MD', 'ttime_assert_PM',
       'ttime_assert_NT', 'lanes_AM', 'lanes_

# Write model network as shapefile

In [46]:
out_cols = ['A', 'B','model_link_id', 'id', 'assign_group', 'drive_access', 'roadway_class',
            'lanes_AM', 'lanes_MD', 'lanes_PM', 'lanes_NT', 'segment_id', 'HOV', 'bike', 'walk','roadway',
            'price_sov_AM', 'geometry', 'managed']

m_net.write_roadway_as_shp(
    output_link_shp = os.path.join(output_dir, 'model_networks', 'shapefiles', 'linksRW2.shp'),
    output_node_shp = os.path.join(output_dir, 'model_networks', 'shapefiles', 'nodesRW2.shp'),
    link_output_variables = out_cols,
    data_to_csv = False,
    data_to_dbf = True,
    export_drive_only = False, # if user only wants drive links/nodes in the shapefile
)

2022-07-08 13:55:59, INFO: Writing Network as Shapefile
2022-07-08 13:55:59, INFO: Writing Network as Shapefile
2022-07-08 13:55:59, INFO: Renaming DBF Node Variables
2022-07-08 13:55:59, INFO: Renaming DBF Node Variables
2022-07-08 13:55:59, INFO: Renaming variables so that they are DBF-safe
2022-07-08 13:55:59, INFO: Renaming variables so that they are DBF-safe
2022-07-08 13:56:14, INFO: Renaming DBF Link Variables
2022-07-08 13:56:14, INFO: Renaming DBF Link Variables
2022-07-08 13:56:14, INFO: Renaming variables so that they are DBF-safe
2022-07-08 13:56:14, INFO: Renaming variables so that they are DBF-safe
2022-07-08 13:56:15, INFO: Writing Node Shapes:
 - C:/MetC_LassoData/data/processed/version_01\model_networks\shapefiles\nodesRW2.shp
2022-07-08 13:56:15, INFO: Writing Node Shapes:
 - C:/MetC_LassoData/data/processed/version_01\model_networks\shapefiles\nodesRW2.shp
2022-07-08 13:57:38, INFO: Writing Link Shapes:
 - C:/MetC_LassoData/data/processed/version_01\model_networks\sh

# Write model network for Cube

In [47]:
m_net.write_roadway_as_fixedwidth(
    output_link_txt = os.path.join(output_dir, 'model_networks', 'links.txt'),
    output_node_txt = os.path.join(output_dir, 'model_networks', 'nodes.txt'),
    output_link_header_width_txt = os.path.join(output_dir, 'model_networks', 'links_header_width.txt'),
    output_node_header_width_txt = os.path.join(output_dir, 'model_networks', 'nodes_header_width.txt'),
    output_cube_network_script = os.path.join(output_dir, 'model_networks', 'make_complete_network_from_fixed_width_file.s'),
)

2022-06-30 11:51:49, INFO: Starting fixed width conversion
2022-06-30 11:51:49, INFO: Starting fixed width conversion
2022-06-30 12:13:29, INFO: Writing out link database
2022-06-30 12:13:29, INFO: Writing out link database
2022-06-30 12:13:56, INFO: Writing out link header and width ----
2022-06-30 12:13:56, INFO: Writing out link header and width ----
2022-06-30 12:13:56, INFO: Starting fixed width conversion
2022-06-30 12:13:56, INFO: Starting fixed width conversion
2022-06-30 12:15:12, INFO: Writing out node database
2022-06-30 12:15:12, INFO: Writing out node database
2022-06-30 12:15:14, INFO: Writing out node header and width
2022-06-30 12:15:14, INFO: Writing out node header and width


In [48]:
version_01_scenario.transit_net.set_roadnet(version_01_scenario.road_net, validate_consistency=False)

AttributeError: module 'osmnx' has no attribute 'gdfs_to_graph'

In [ ]:
standard_transit_net = StandardTransit.fromTransitNetwork(version_01_scenario.transit_net, parameters=parameters)
standard_transit_net.write_as_cube_lin(outpath = os.path.join(output_dir, 'model_networks', "transit.lin"))

ModuleNotFoundError: No module named 'packagename'

In [61]:
version_00_scenario.road_net.nodes_df.model_node_id.max()

417322

In [67]:
version_00_scenario.road_net.nodes_df.model_node_id.shape

(414222,)

In [68]:
version_01_scenario.road_net.nodes_df.model_node_id.max()

417322

In [69]:
version_01_scenario.road_net.nodes_df.model_node_id.shape

(414253,)

In [70]:
m_net.nodes_df.model_node_id.max()

417322

In [71]:
m_net.nodes_df.model_node_id.shape

(417283,)